#Import the required libraries




In [ ]:
#! pip install -r requirements.txt

In [1]:
!pip install faiss-gpu
!pip install pypdf
!pip install -U langchain-community
!pip install tiktoken
!pip install --upgrade gradio

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 26.0 MB/s eta 0:00:00:00:0100:01
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 5.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.3/386.3 kB 4.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 611.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 1.0 MB/s eta 0:00:00eta 0:00:01
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsm

In [2]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA, RetrievalQA

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader, PyPDFLoader
from langchain.vectorstores import FAISS
import openai
import os
import faiss

# Loading Documents


In [3]:
loader = PyPDFLoader('https://www.nestle.com/sites/default/files/asset-library/documents/jobs/the_nestle_hr_policy_pdf_2012.pdf')
documents = loader.load()

 # Creating Vector Representation of Texts

In [4]:
# Splitting documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Generating embeddings
embeddings = OpenAIEmbeddings()

# Using FAISS as the vector database
vectordb = FAISS.from_documents(texts, embeddings)

/tmp/ipykernel_116/1191152576.py:6: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [5]:
#import os
#os.environ["OPENAI_API_KEY"] = "YOUR KEY"  # Replace with your actual API key

# Setting Up Question-Answering System

In [6]:
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())

/tmp/ipykernel_116/3946707704.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())


# Defining Prompt Template

In [7]:
from langchain import PromptTemplate

# Define the prompt template in English
template = """
I am a HR helpful assistant. Please answer the following question in English.
Question: {question}
Answer:
"""

# Create the PromptTemplate instance with the modified English template
prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)


In [8]:
# Example usage
example_question = "What are the best practices for conducting interviews?"
formatted_prompt = prompt.format(question=example_question)

# Print the result to see the formatted prompt
print(formatted_prompt)



I am a HR helpful assistant. Please answer the following question in English.
Question: What are the best practices for conducting interviews?
Answer:



In [9]:
from langchain.chains import LLMChain  # Import LLMChain
from langchain.chat_models import ChatOpenAI # Import ChatOpenAI

# Use ChatOpenAI for chat models instead of OpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Create the LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Example usage
example_question = "What are the best practices for conducting interviews?"
response = llm_chain.run(question=example_question)

# Print the result
print(response)

/tmp/ipykernel_116/990803728.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)
/tmp/ipykernel_116/990803728.py:12: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm_chain.run(question=example_question)


There are several best practices for conducting interviews, including:

1. Prepare for the interview by reviewing the candidate's resume and job description.
2. Create a structured interview process with a set list of questions.
3. Be mindful of the candidate's time and schedule the interview at a convenient time for them.
4. Create a welcoming and professional atmosphere for the interview.
5. Ask open-ended questions to encourage the candidate to provide detailed responses.
6. Listen actively and take notes during the interview.
7. Avoid asking illegal or discriminatory questions.
8. Provide the candidate with information about the company and the role.
9. Allow the candidate to ask questions about the role and company.
10. Follow up with the candidate after the interview to provide feedback and next steps.


#  Building Chat Interface with Gradio and Launching the Chat Interface

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Test code
prompt = PromptTemplate(
    input_variables=["question"],
    template="I am a helpful HR assistant. Answer in English. Question: {question} Answer:",
)

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Test with a sample question
response = llm_chain.run(question="What are the best practices for conducting interviews?")
print(f"Response: {response}")


Response: 1. Prepare thoroughly: Review the candidate's resume and job description, and prepare a list of relevant questions.

2. Create a comfortable environment: Ensure the interview room is quiet, well-lit, and free from distractions.

3. Use a structured interview format: Ask each candidate the same set of questions to ensure consistency and fairness.

4. Listen actively: Pay attention to the candidate's responses and ask follow-up questions to gather more information.

5. Take notes: Document key points from the interview to help with decision-making later on.

6. Be respectful and professional: Treat all candidates with courtesy and respect, regardless of their qualifications.

7. Provide clear information: Explain the next steps in the hiring process and answer any questions the candidate may have.

8. Follow up promptly: Contact all candidates to inform them of the hiring decision, whether they were successful or not.


In [ ]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Set OpenAI API Key
#os.environ["OPENAI_API_KEY"] = "YOUR KEY"

# Define the prompt template
template = """
I am a helpful assistant. Please answer the following question in English.
Question: {question}
Answer:
"""
prompt = PromptTemplate(input_variables=["question"], template=template)

# Initialize ChatOpenAI and LLMChain
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)
llm_chain = LLMChain(prompt=prompt, llm=llm)

# Gradio bot function
def bot(history):
    try:
        # Ensure history is initialized
        if not history:
            history = []

        # Get the user's query
        query = history[-1][0]  # Last user input
        print(f"User query: {query}")  # Debug log

        # Send the query to LangChain's LLMChain
        response = llm_chain.run(question=query)
        print(f"LangChain response: {response}")  # Debug log

        # Update chatbot history with the response
        history[-1] = (query, response)
        return history
#end of try:
    except Exception as e:
        # Log the error
        print(f"Error occurred: {e}")

        # Update chatbot history with an error message
        if history:
            history[-1] = (history[-1][0], "An error occurred. Please try again later.")
        return history or [("Error", "An unexpected error occurred. Please try again later.")]

# Gradio Interface
with gr.Blocks() as demo:
    chatbot = gr.Chatbot(value=[], label="Chat with LangChain")
    txt = gr.Textbox(show_label=False, placeholder="Type your question here...")

    txt.submit(
        lambda history, text: (history + [(text, None)], ""),  # Append user input
        [chatbot, txt],
        [chatbot, txt],
    ).then(bot, chatbot, chatbot)

demo.launch(debug=True, share=True)

/tmp/ipykernel_116/2410043725.py:52: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(value=[], label="Chat with LangChain")


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://59c7176f74253726c5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


User query: Hi
LangChain response: Hello! How can I assist you today?
